# Notebook 01: Data Cleaning + Preprocessing Pipeline

**Project:** Vehicle Sales & Market Insights  
**Dataset:** car_prices_interim.csv (from Notebook 00)  
**Purpose:** Clean data quality issues and build reusable preprocessing pipeline

## Objective
Address data quality issues identified in Notebook 00 and create production-ready cleaned dataset:
- Standardize categorical variables
- Handle outliers
- Impute missing values strategically
- Engineer essential features
- Save preprocessed data for modeling

## Approach
We will NOT drop records with missing temporal data. Instead, we'll use a has_date flag to preserve all 558,837 records, maximizing training data while handling missing patterns appropriately.

## Step 1: Environment Setup
Load necessary libraries and the interim dataset.

In [29]:
# Data manipulation
import pandas as pd
import numpy as np

# Preprocessing & pipelines
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Utilities
import pickle
import warnings
import os

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)
warnings.filterwarnings('ignore')

print("Environment Setup Complete")
print(f"Pandas: {pd.__version__}")
print(f"NumPy: {np.__version__}")

# Load interim data from Notebook 00
df = pd.read_csv('data/interim/car_prices_interim.csv')

print(f"\nDataset loaded: {df.shape}")
print(f"Memory: {df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")
print(f"\nColumns: {list(df.columns)}")

Environment Setup Complete
Pandas: 2.3.3
NumPy: 1.26.4

Dataset loaded: (558837, 23)
Memory: 432.5 MB

Columns: ['year', 'make', 'model', 'trim', 'body', 'transmission', 'vin', 'state', 'condition', 'odometer', 'color', 'interior', 'seller', 'mmr', 'sellingprice', 'saledate', 'sale_year', 'sale_month', 'sale_day', 'sale_dayofweek', 'sale_quarter', 'vehicle_age', 'price_diff']


## Step 2: Baseline Data Quality Assessment

Review data quality issues identified in Notebook 00 to establish cleaning priorities.
Document current state before applying transformations.

In [30]:
print("BASELINE DATA QUALITY ASSESSMENT")
print("=" * 80)

# Missing values
print("1. Missing Values:")
missing = df.isnull().sum()
missing_df = pd.DataFrame({
    'Missing': missing[missing > 0],
    'Percent': (missing[missing > 0] / len(df) * 100).round(2)
}).sort_values('Percent', ascending=False)
print(missing_df)

# Known quality issues from Notebook 00
print("\n2. Known Data Quality Issues:")
print(f"   - Body type unique values: {df['body'].nunique()} (capitalization inconsistency)")
print(f"   - Transmission unique values: {df['transmission'].nunique()}")

# Check invalid transmission
invalid_trans = df[~df['transmission'].isin(['automatic', 'manual'])]['transmission'].value_counts()
if len(invalid_trans) > 0:
    print(f"   - Invalid transmission entries:")
    print(invalid_trans.to_string().replace('\n', '\n     '))

print(f"   - Odometer outliers (>500k): {(df['odometer'] > 500000).sum()}")
print(f"   - Duplicate VINs: {df['vin'].duplicated().sum()}")

# Target variable
print("\n3. Target Variable (sellingprice):")
print(f"   - Missing: {df['sellingprice'].isnull().sum()}")
print(f"   - Range: ${df['sellingprice'].min():.0f} to ${df['sellingprice'].max():.0f}")
print(f"   - Mean: ${df['sellingprice'].mean():.2f}")

print("\n4. Temporal Features:")
print(f"   - Records with valid dates: {df['saledate'].notna().sum():,} ({df['saledate'].notna().sum()/len(df)*100:.1f}%)")
print(f"   - Records without dates: {df['saledate'].isna().sum():,} ({df['saledate'].isna().sum()/len(df)*100:.1f}%)")

print("\n" + "=" * 80)
print("Data Cleaning Plan:")
print("  1. Remove records with missing target (sellingprice)")
print("  2. Standardize categorical variables (fix capitalization)")
print("  3. Fix invalid transmission values")
print("  4. Handle odometer outliers (cap at 500k)")
print("  5. Create has_date flag, drop derived temporal features")
print("  6. Impute remaining missing values")

BASELINE DATA QUALITY ASSESSMENT
1. Missing Values:
                Missing  Percent
sale_dayofweek   163348    29.23
vehicle_age      163348    29.23
sale_quarter     163348    29.23
saledate         163348    29.23
sale_year        163348    29.23
sale_month       163348    29.23
sale_day         163348    29.23
transmission      65352    11.69
body              13195     2.36
condition         11820     2.12
trim              10651     1.91
model             10399     1.86
make              10301     1.84
color               749     0.13
interior            749     0.13
odometer             94     0.02
mmr                  38     0.01
price_diff           38     0.01
sellingprice         12     0.00
vin                   4     0.00

2. Known Data Quality Issues:
   - Body type unique values: 87 (capitalization inconsistency)
   - Transmission unique values: 4
   - Invalid transmission entries:
transmission
     sedan    15
     Sedan    11
   - Odometer outliers (>500k): 81
   - Dup

## Step 3: Remove Records with Missing Target Variable

Remove records where sellingprice is missing (12 records).
These cannot be used for model training and represent only 0.002% of data.

In [31]:
print("REMOVING RECORDS WITH MISSING TARGET")
print("=" * 80)

print(f"Before: {len(df):,} records")
print(f"Missing sellingprice: {df['sellingprice'].isnull().sum()}")

# Remove records with missing target
df = df[df['sellingprice'].notna()].copy()

print(f"\nAfter: {len(df):,} records")
print(f"Removed: 12 records (0.002%)")
print(f"Retention rate: 99.998%")

# Verify
print(f"\nVerification:")
print(f"Missing sellingprice: {df['sellingprice'].isnull().sum()}")
print(f"Mean: ${df['sellingprice'].mean():.2f}")
print(f"Median: ${df['sellingprice'].median():.2f}")

REMOVING RECORDS WITH MISSING TARGET
Before: 558,837 records
Missing sellingprice: 12

After: 558,825 records
Removed: 12 records (0.002%)
Retention rate: 99.998%

Verification:
Missing sellingprice: 0
Mean: $13611.36
Median: $12100.00


## Step 4: Standardize Categorical Variables

Fix inconsistent capitalization in categorical text fields.
Convert to title case for consistency and reduce cardinality.

In [32]:
print("CATEGORICAL VARIABLE STANDARDIZATION")
print("=" * 80)

# Categorical columns to standardize
cat_columns = ['make', 'model', 'body', 'transmission', 'color', 'interior', 'state']

print("Before Standardization:")
for col in cat_columns:
    print(f"  {col}: {df[col].nunique()} unique values")

# Apply title case standardization
for col in cat_columns:
    df[col] = df[col].astype(str).str.strip().str.title()

print("\nAfter Standardization:")
for col in cat_columns:
    print(f"  {col}: {df[col].nunique()} unique values")

# Check specific improvements
print("\nBody Type - Top 10:")
print(df['body'].value_counts().head(10))

print("\nTransmission Values:")
print(df['transmission'].value_counts())

# Check for 'Nan' strings created by conversion
print("\nChecking for 'Nan' string conversions:")
nan_counts = {}
for col in cat_columns:
    nan_count = (df[col] == 'Nan').sum()
    if nan_count > 0:
        nan_counts[col] = nan_count
        
if nan_counts:
    for col, count in nan_counts.items():
        print(f"  {col}: {count:,} 'Nan' strings")
else:
    print("  No 'Nan' strings found")

CATEGORICAL VARIABLE STANDARDIZATION
Before Standardization:
  make: 96 unique values
  model: 973 unique values
  body: 87 unique values
  transmission: 4 unique values
  color: 46 unique values
  interior: 17 unique values
  state: 64 unique values

After Standardization:
  make: 67 unique values
  model: 852 unique values
  body: 47 unique values
  transmission: 4 unique values
  color: 47 unique values
  interior: 18 unique values
  state: 64 unique values

Body Type - Top 10:
body
Sedan          241332
Suv            143844
Hatchback       26237
Minivan         25529
Coupe           17752
Crew Cab        16394
Wagon           16128
Nan             13195
Convertible     10476
Supercrew        9033
Name: count, dtype: int64

Transmission Values:
transmission
Automatic    475904
Nan           65351
Manual        17544
Sedan            26
Name: count, dtype: int64

Checking for 'Nan' string conversions:
  make: 10,301 'Nan' strings
  model: 10,399 'Nan' strings
  body: 13,195 'Nan' st

## Step 5: Fix Missing Value Encoding and Invalid Transmission

Convert 'Nan' strings back to proper NaN values.
Remove invalid 'Sedan' entries from transmission field.

In [33]:
print("FIXING MISSING VALUES AND INVALID DATA")
print("=" * 80)

cat_columns = ['make', 'model', 'body', 'transmission', 'color', 'interior']

# Convert 'Nan' strings to actual NaN
print("Converting 'Nan' strings to NaN:")
for col in cat_columns:
    before = (df[col] == 'Nan').sum()
    df[col] = df[col].replace('Nan', np.nan)
    after = df[col].isnull().sum()
    print(f"  {col}: {before:,} 'Nan' strings → {after:,} NaN values")

# Fix invalid transmission values
print("\nFixing Invalid Transmission Values:")
print(f"Before:")
print(df['transmission'].value_counts(dropna=False).to_string().replace('\n', '\n  '))

invalid_count = (df['transmission'] == 'Sedan').sum()
df.loc[df['transmission'] == 'Sedan', 'transmission'] = np.nan

print(f"\nAfter (converted {invalid_count} 'Sedan' to NaN):")
print(df['transmission'].value_counts(dropna=False).to_string().replace('\n', '\n  '))

print("\n" + "=" * 80)
print("Summary:")
print(f"Valid transmission values: {df['transmission'].notna().sum():,}")
print(f"Body types (excl. missing): {df['body'].nunique()}")
print(f"Total records: {len(df):,}")

FIXING MISSING VALUES AND INVALID DATA
Converting 'Nan' strings to NaN:
  make: 10,301 'Nan' strings → 10,301 NaN values
  model: 10,399 'Nan' strings → 10,399 NaN values
  body: 13,195 'Nan' strings → 13,195 NaN values
  transmission: 65,351 'Nan' strings → 65,351 NaN values
  color: 749 'Nan' strings → 749 NaN values
  interior: 749 'Nan' strings → 749 NaN values

Fixing Invalid Transmission Values:
Before:
transmission
  Automatic    475904
  NaN           65351
  Manual        17544
  Sedan            26

After (converted 26 'Sedan' to NaN):
transmission
  Automatic    475904
  NaN           65377
  Manual        17544

Summary:
Valid transmission values: 493,448
Body types (excl. missing): 46
Total records: 558,825


## Step 6: Handle Outliers in Numeric Features

Cap extreme odometer outliers at 500,000 miles.
Preserve price outliers as they represent legitimate luxury vehicles.

In [35]:
print("OUTLIER HANDLING")
print("=" * 80)

# Analyze odometer distribution
print("Odometer Analysis:")
print(f"  Mean: {df['odometer'].mean():,.0f}")
print(f"  Median: {df['odometer'].median():,.0f}")
print(f"  99th percentile: {df['odometer'].quantile(0.99):,.0f}")
print(f"  Max: {df['odometer'].max():,.0f}")

print("\n  Outlier counts:")
print(f"    >200k miles: {(df['odometer'] > 200000).sum():,}")
print(f"    >300k miles: {(df['odometer'] > 300000).sum():,}")
print(f"    >400k miles: {(df['odometer'] > 400000).sum():,}")
print(f"    >500k miles: {(df['odometer'] > 500000).sum():,}")

# Apply capping
odometer_cap = 500000
outliers_capped = (df['odometer'] > odometer_cap).sum()
df['odometer'] = df['odometer'].clip(upper=odometer_cap)

print(f"\nApplied cap at {odometer_cap:,} miles")
print(f"  Records capped: {outliers_capped}")
print(f"  New max: {df['odometer'].max():,.0f}")

# Check other numeric features
print("\n" + "-" * 80)
print("Other Numeric Features (checking for extreme outliers):")

for col in ['condition', 'mmr', 'sellingprice']:
    q1 = df[col].quantile(0.25)
    q3 = df[col].quantile(0.75)
    iqr = q3 - q1
    outliers = ((df[col] < q1 - 3*iqr) | (df[col] > q3 + 3*iqr)).sum()
    print(f"  {col}: {outliers:,} outliers beyond 3×IQR")

print("\nDecision: Keep price outliers (legitimate luxury vehicles)")
print("          Condition scale (1-49) is valid as-is")

OUTLIER HANDLING
Odometer Analysis:
  Mean: 68,321
  Median: 52,255
  99th percentile: 226,670
  Max: 999,999

  Outlier counts:
    >200k miles: 12,111
    >300k miles: 744
    >400k miles: 123
    >500k miles: 81

Applied cap at 500,000 miles
  Records capped: 81
  New max: 500,000

--------------------------------------------------------------------------------
Other Numeric Features (checking for extreme outliers):
  condition: 0 outliers beyond 3×IQR
  mmr: 3,029 outliers beyond 3×IQR
  sellingprice: 2,927 outliers beyond 3×IQR

Decision: Keep price outliers (legitimate luxury vehicles)
          Condition scale (1-49) is valid as-is


## Step 7: Handle Temporal Features

Create has_date binary flag to capture temporal data availability.
Drop derived temporal features since they show weak correlation with price and 29% are missing.
This preserves all records while avoiding artificial imputation.

In [36]:
print("TEMPORAL FEATURE HANDLING")
print("=" * 80)

# Create has_date flag
df['has_date'] = df['saledate'].notna().astype(int)

print(f"Records with dates: {df['has_date'].sum():,} ({df['has_date'].mean()*100:.1f}%)")
print(f"Records without dates: {(1-df['has_date']).sum():,} ({(1-df['has_date'].mean())*100:.1f}%)")

# Analyze price difference
print(f"\nPrice Analysis by Date Availability:")
print(f"  Mean price WITH dates:    ${df[df['has_date']==1]['sellingprice'].mean():,.2f}")
print(f"  Mean price WITHOUT dates: ${df[df['has_date']==0]['sellingprice'].mean():,.2f}")
print(f"  Difference: ${df[df['has_date']==0]['sellingprice'].mean() - df[df['has_date']==1]['sellingprice'].mean():,.2f}")

# Drop derived temporal features
temporal_to_drop = ['sale_year', 'sale_month', 'sale_day', 'sale_dayofweek', 
                    'sale_quarter', 'vehicle_age', 'price_diff']

print(f"\nDropping {len(temporal_to_drop)} derived temporal features:")
for col in temporal_to_drop:
    print(f"  - {col}")

df = df.drop(columns=temporal_to_drop)

print(f"\nNew shape: {df.shape}")
print(f"Columns retained: {len(df.columns)}")
print(f"All {len(df):,} records preserved")

TEMPORAL FEATURE HANDLING
Records with dates: 395,489 (70.8%)
Records without dates: 163,336 (29.2%)

Price Analysis by Date Availability:
  Mean price WITH dates:    $13,176.61
  Mean price WITHOUT dates: $14,664.03
  Difference: $1,487.42

Dropping 7 derived temporal features:
  - sale_year
  - sale_month
  - sale_day
  - sale_dayofweek
  - sale_quarter
  - vehicle_age
  - price_diff

New shape: (558825, 17)
Columns retained: 17
All 558,825 records preserved


## Step 8: Missing Value Imputation

Apply appropriate imputation strategies for remaining missing values:
- Categorical: Mode imputation or 'Unknown' category
- Numeric: Median imputation

In [37]:
print("MISSING VALUE IMPUTATION")
print("=" * 80)

print("Before Imputation:")
missing = df.isnull().sum()
missing_df = pd.DataFrame({
    'Missing': missing[missing > 0],
    'Percent': (missing[missing > 0] / len(df) * 100).round(2)
}).sort_values('Percent', ascending=False)
print(missing_df)

print("\n" + "-" * 80)
print("Applying Imputation:\n")

# Transmission - Mode
transmission_mode = df['transmission'].mode()[0]
n_trans = df['transmission'].isnull().sum()
df['transmission'].fillna(transmission_mode, inplace=True)
print(f"1. Transmission: {n_trans:,} → '{transmission_mode}'")

# Body - Mode
body_mode = df['body'].mode()[0]
n_body = df['body'].isnull().sum()
df['body'].fillna(body_mode, inplace=True)
print(f"2. Body: {n_body:,} → '{body_mode}'")

# Condition - Median
condition_median = df['condition'].median()
n_cond = df['condition'].isnull().sum()
df['condition'].fillna(condition_median, inplace=True)
print(f"3. Condition: {n_cond:,} → {condition_median}")

# Make/Model/Trim - 'Unknown'
n_make = df['make'].isnull().sum()
n_model = df['model'].isnull().sum()
n_trim = df['trim'].isnull().sum()
df['make'].fillna('Unknown', inplace=True)
df['model'].fillna('Unknown', inplace=True)
df['trim'].fillna('Unknown', inplace=True)
print(f"4. Make: {n_make:,} → 'Unknown'")
print(f"5. Model: {n_model:,} → 'Unknown'")
print(f"6. Trim: {n_trim:,} → 'Unknown'")

# Color/Interior - Mode
color_mode = df['color'].mode()[0]
interior_mode = df['interior'].mode()[0]
n_color = df['color'].isnull().sum()
n_interior = df['interior'].isnull().sum()
df['color'].fillna(color_mode, inplace=True)
df['interior'].fillna(interior_mode, inplace=True)
print(f"7. Color: {n_color:,} → '{color_mode}'")
print(f"8. Interior: {n_interior:,} → '{interior_mode}'")

# Odometer - Median
n_odo = df['odometer'].isnull().sum()
if n_odo > 0:
    odometer_median = df['odometer'].median()
    df['odometer'].fillna(odometer_median, inplace=True)
    print(f"9. Odometer: {n_odo:,} → {odometer_median:,.0f}")

# MMR - Median
n_mmr = df['mmr'].isnull().sum()
if n_mmr > 0:
    mmr_median = df['mmr'].median()
    df['mmr'].fillna(mmr_median, inplace=True)
    print(f"10. MMR: {n_mmr:,} → ${mmr_median:,.0f}")

# VIN - Placeholder (not used for modeling)
n_vin = df['vin'].isnull().sum()
if n_vin > 0:
    df['vin'].fillna('UNKNOWN_VIN', inplace=True)
    print(f"11. VIN: {n_vin:,} → 'UNKNOWN_VIN'")

print("\n" + "=" * 80)
print("After Imputation:")
remaining = df.isnull().sum()[df.isnull().sum() > 0]
if len(remaining) == 0:
    print("All features fully imputed!")
elif len(remaining) == 1 and 'saledate' in remaining.index:
    print(f"Only saledate has {remaining['saledate']:,} missing (intentionally kept)")
else:
    print(remaining)

print(f"\nData quality: {(1 - df.isnull().sum().sum()/(len(df)*len(df.columns)))*100:.2f}% complete")

MISSING VALUE IMPUTATION
Before Imputation:
              Missing  Percent
saledate       163336    29.23
transmission    65377    11.70
body            13195     2.36
condition       11820     2.12
trim            10651     1.91
model           10399     1.86
make            10301     1.84
color             749     0.13
interior          749     0.13
odometer           94     0.02
vin                 4     0.00
mmr                26     0.00

--------------------------------------------------------------------------------
Applying Imputation:

1. Transmission: 65,377 → 'Automatic'
2. Body: 13,195 → 'Sedan'
3. Condition: 11,820 → 35.0
4. Make: 10,301 → 'Unknown'
5. Model: 10,399 → 'Unknown'
6. Trim: 10,651 → 'Unknown'
7. Color: 749 → 'Black'
8. Interior: 749 → 'Black'
9. Odometer: 94 → 52,255
10. MMR: 26 → $12,250
11. VIN: 4 → 'UNKNOWN_VIN'

After Imputation:
Only saledate has 163,336 missing (intentionally kept)

Data quality: 98.28% complete


## Step 9: Feature Engineering

Create essential features for modeling:
- Vehicle age from current year
- Log-transformed odometer (handles skewness)
- Price difference (selling price vs MMR)
- MMR ratio (selling/MMR)

In [39]:
print("FEATURE ENGINEERING")
print("=" * 80)

# 1. Vehicle Age (based on year 2015 - last sale date in dataset)
reference_year = 2015
df['vehicle_age'] = reference_year - df['year']
print(f"1. Vehicle Age:")
print(f"   Range: {df['vehicle_age'].min()} to {df['vehicle_age'].max()} years")
print(f"   Mean: {df['vehicle_age'].mean():.1f} years")

# 2. Log Odometer (handle skewed distribution)
df['log_odometer'] = np.log1p(df['odometer'])
print(f"\n2. Log Odometer:")
print(f"   Range: {df['log_odometer'].min():.2f} to {df['log_odometer'].max():.2f}")
print(f"   Mean: {df['log_odometer'].mean():.2f}")

# 3. Price Difference (selling price - MMR)
df['price_diff'] = df['sellingprice'] - df['mmr']
print(f"\n3. Price Difference (selling - MMR):")
print(f"   Mean: ${df['price_diff'].mean():.2f}")
print(f"   Median: ${df['price_diff'].median():.2f}")
print(f"   Negative (sold below MMR): {(df['price_diff'] < 0).sum():,} ({(df['price_diff'] < 0).mean()*100:.1f}%)")

# 4. MMR Ratio (selling / MMR)
df['mmr_ratio'] = df['sellingprice'] / df['mmr']
print(f"\n4. MMR Ratio:")
print(f"   Mean: {df['mmr_ratio'].mean():.3f}")
print(f"   Median: {df['mmr_ratio'].median():.3f}")
print(f"   Sold above MMR: {(df['mmr_ratio'] > 1).sum():,} ({(df['mmr_ratio'] > 1).mean()*100:.1f}%)")

# 5. Age-Odometer Interaction (captures high-mileage vs age relationship)
df['age_odo_interaction'] = df['vehicle_age'] * df['odometer'] / 10000
print(f"\n5. Age-Odometer Interaction:")
print(f"   Mean: {df['age_odo_interaction'].mean():.1f}")
print(f"   Median: {df['age_odo_interaction'].median():.1f}")

print("\n" + "=" * 80)
print("Feature Engineering Complete:")
print(f"New features created: 5")
print(f"Total features: {len(df.columns)}")
print(f"Dataset shape: {df.shape}")

FEATURE ENGINEERING
1. Vehicle Age:
   Range: 0 to 33 years
   Mean: 5.0 years

2. Log Odometer:
   Range: 0.69 to 13.12
   Mean: 10.78

3. Price Difference (selling - MMR):
   Mean: $-157.95
   Median: $-50.00
   Negative (sold below MMR): 286,415 (51.3%)

4. MMR Ratio:
   Mean: 0.993
   Median: 0.996
   Sold above MMR: 261,187 (46.7%)

5. Age-Odometer Interaction:
   Mean: 50.2
   Median: 15.7

Feature Engineering Complete:
New features created: 5
Total features: 22
Dataset shape: (558825, 22)


## Step 10: Handle High-Cardinality Categoricals

Reduce cardinality in features with many unique values by grouping rare categories.
Focus on seller, model, and trim which have thousands of unique values.

In [40]:
print("HIGH-CARDINALITY CATEGORICAL HANDLING")
print("=" * 80)

# Analyze cardinality
cat_features = ['make', 'model', 'trim', 'body', 'transmission', 'state', 
                'color', 'interior', 'seller']

print("Current Cardinality:")
for col in cat_features:
    print(f"  {col}: {df[col].nunique():,} unique values")

print("\n" + "-" * 80)
print("Grouping Strategy:\n")

# 1. Seller - Keep top 100, group rest as 'Other'
seller_threshold = 100
top_sellers = df['seller'].value_counts().head(seller_threshold).index
df['seller_grouped'] = df['seller'].apply(lambda x: x if x in top_sellers else 'Other_Seller')
print(f"1. Seller: Top {seller_threshold} kept, rest → 'Other_Seller'")
print(f"   Before: {df['seller'].nunique():,} | After: {df['seller_grouped'].nunique():,}")

# 2. Model - Keep top 200, group rest as 'Other'
model_threshold = 200
top_models = df['model'].value_counts().head(model_threshold).index
df['model_grouped'] = df['model'].apply(lambda x: x if x in top_models else 'Other_Model')
print(f"\n2. Model: Top {model_threshold} kept, rest → 'Other_Model'")
print(f"   Before: {df['model'].nunique():,} | After: {df['model_grouped'].nunique():,}")

# 3. Trim - Keep top 100, group rest as 'Other'
trim_threshold = 100
top_trims = df['trim'].value_counts().head(trim_threshold).index
df['trim_grouped'] = df['trim'].apply(lambda x: x if x in top_trims else 'Other_Trim')
print(f"\n3. Trim: Top {trim_threshold} kept, rest → 'Other_Trim'")
print(f"   Before: {df['trim'].nunique():,} | After: {df['trim_grouped'].nunique():,}")

# Drop original high-cardinality columns
df = df.drop(columns=['seller', 'model', 'trim'])

print("\n" + "=" * 80)
print("High-Cardinality Handling Complete:")
print(f"Columns dropped: seller, model, trim")
print(f"Columns added: seller_grouped, model_grouped, trim_grouped")
print(f"New shape: {df.shape}")

HIGH-CARDINALITY CATEGORICAL HANDLING
Current Cardinality:
  make: 67 unique values
  model: 852 unique values
  trim: 1,964 unique values
  body: 46 unique values
  transmission: 2 unique values
  state: 64 unique values
  color: 46 unique values
  interior: 17 unique values
  seller: 14,262 unique values

--------------------------------------------------------------------------------
Grouping Strategy:

1. Seller: Top 100 kept, rest → 'Other_Seller'
   Before: 14,262 | After: 101

2. Model: Top 200 kept, rest → 'Other_Model'
   Before: 852 | After: 201

3. Trim: Top 100 kept, rest → 'Other_Trim'
   Before: 1,964 | After: 101

High-Cardinality Handling Complete:
Columns dropped: seller, model, trim
Columns added: seller_grouped, model_grouped, trim_grouped
New shape: (558825, 22)


## Step 11: Save Cleaned Dataset

Export the fully cleaned and feature-engineered dataset for modeling.
Document final data quality and feature set.

In [41]:
print("SAVING CLEANED DATASET")
print("=" * 80)

# Final data quality check
print("Final Data Quality Summary:")
print(f"Total records: {len(df):,}")
print(f"Total features: {len(df.columns)}")
print(f"Missing values: {df.isnull().sum().sum():,}")
missing_cols = df.isnull().sum()[df.isnull().sum() > 0]
if len(missing_cols) > 0:
    print(f"Columns with missing: {list(missing_cols.index)}")
else:
    print("All modeling features complete!")

# Feature summary
print("\n" + "-" * 80)
print("Feature Categories:")

numeric_features = ['year', 'condition', 'odometer', 'mmr', 'sellingprice', 
                    'vehicle_age', 'log_odometer', 'price_diff', 'mmr_ratio', 
                    'age_odo_interaction', 'has_date']
categorical_features = ['make', 'body', 'transmission', 'state', 'color', 
                       'interior', 'seller_grouped', 'model_grouped', 'trim_grouped']
id_features = ['vin', 'saledate']

print(f"Numeric features ({len(numeric_features)}): {numeric_features}")
print(f"Categorical features ({len(categorical_features)}): {categorical_features}")
print(f"ID/Reference ({len(id_features)}): {id_features}")

# Save cleaned dataset
output_path = 'data/processed/car_prices_cleaned.csv'
df.to_csv(output_path, index=False)

print("\n" + "=" * 80)
print(f"Dataset saved: {output_path}")
print(f"File size: {os.path.getsize(output_path) / 1024**2:.1f} MB")

# Summary statistics
print("\n" + "-" * 80)
print("Cleaning Summary:")
print(f"  Records retained: 558,825 / 558,837 (99.998%)")
print(f"  Features created: 5 (vehicle_age, log_odometer, price_diff, mmr_ratio, age_odo_interaction)")
print(f"  High-cardinality reduced: seller (14,262→101), model (852→201), trim (1,964→101)")
print(f"  Outliers capped: 81 odometer values")
print(f"  Missing values imputed: transmission, body, condition, make, model, colors")
print(f"  Data completeness: 98.3%")

print("\n" + "=" * 80)
print("Notebook 01 Complete!")
print("Next: Notebook 02 - Modeling + Evaluation + Hyperparameter Tuning")

SAVING CLEANED DATASET
Final Data Quality Summary:
Total records: 558,825
Total features: 22
Missing values: 163,336
Columns with missing: ['saledate']

--------------------------------------------------------------------------------
Feature Categories:
Numeric features (11): ['year', 'condition', 'odometer', 'mmr', 'sellingprice', 'vehicle_age', 'log_odometer', 'price_diff', 'mmr_ratio', 'age_odo_interaction', 'has_date']
Categorical features (9): ['make', 'body', 'transmission', 'state', 'color', 'interior', 'seller_grouped', 'model_grouped', 'trim_grouped']
ID/Reference (2): ['vin', 'saledate']

Dataset saved: data/processed/car_prices_cleaned.csv
File size: 105.1 MB

--------------------------------------------------------------------------------
Cleaning Summary:
  Records retained: 558,825 / 558,837 (99.998%)
  Features created: 5 (vehicle_age, log_odometer, price_diff, mmr_ratio, age_odo_interaction)
  High-cardinality reduced: seller (14,262→101), model (852→201), trim (1,964→1